In [9]:
!pip install kaggle

Defaulting to user installation because normal site-packages is not writeable


In [10]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

The syntax of the command is incorrect.
'cp' is not recognized as an internal or external command,
operable program or batch file.
'chmod' is not recognized as an internal or external command,
operable program or batch file.


In [11]:
# API to fetch the dataset from kaggle

In [12]:
!kaggle datasets download -d kazanova/sentiment140

Traceback (most recent call last):
  File "C:\Program Files\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Program Files\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\Chandan Malakar\AppData\Roaming\Python\Python310\Scripts\kaggle.exe\__main__.py", line 7, in <module>
  File "C:\Users\Chandan Malakar\AppData\Roaming\Python\Python310\site-packages\kaggle\cli.py", line 68, in main
    out = args.func(**command_args)
  File "C:\Users\Chandan Malakar\AppData\Roaming\Python\Python310\site-packages\kaggle\api\kaggle_api_extended.py", line 1741, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
  File "C:\Users\Chandan Malakar\AppData\Roaming\Python\Python310\site-packages\kaggle\api\kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.config_values['username'],
KeyError: 'username'


In [13]:
# Extracting the compressed dataset

In [14]:
!unzip sentiment140.zip

'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [15]:
# Importing the dependencies for the Projects

In [16]:
import numpy as np
import pandas as pd
import re # for regular expresssion - pattern matching
from nltk.corpus import stopwords # for removing the un-influential words from the data, as they're of no use.
from nltk.stem.porter import PorterStemmer # reducig the words to its root words
from sklearn.feature_extraction.text import TfidfVectorizer # for converting the textual data into numerical form
from sklearn.model_selection import train_test_split # for splitting the training data into 3 parts : train, test and split.
from sklearn.linear_model import LogisticRegression # using the Logistic Regression ML model
from sklearn.metrics import accuracy_score # for determining the accuracy

In [17]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Chandan
[nltk_data]     Malakar\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [18]:
# Printing the stopwords in English
print(stopwords.words('english'))
# stopwords are those words in a sentence which doesn't carries any influential meaning to the data
# hence these words will be removed from the data as they're not useful

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

Data Processing

In [19]:
# loading the data

In [21]:
twitter_data = pd.read_csv('twitter_data.csv', encoding='ISO-8859-1')

In [22]:
# checking its shape
twitter_data.shape

(1599999, 6)

In [23]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [24]:
# since there's no col names, manually writing the col names
columns=['target', 'id', 'date', 'flag', 'user', 'text']
twitter_data = pd.read_csv('twitter_data.csv',names = columns, encoding='ISO-8859-1')

In [25]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [26]:
# Counting the number of missing values in the dataset
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [27]:
# checking the distribution of the target column
twitter_data['target'].value_counts()
# here 0 -> means -ve
# and 4 -> means +ve

target
0    800000
4    800000
Name: count, dtype: int64

In [28]:
# converting 4 to 1 for better understanding
twitter_data.replace({'target':{4:1}},inplace=True)

In [29]:
twitter_data['target'].value_counts()

target
0    800000
1    800000
Name: count, dtype: int64

0 --> Negative Tweet

1 --> Positive Tweet

# **Stemming**

# Stemming is the process of reducing a words to its key word or root-word

In [ ]:
# example : actor, actress, acting (words with similar meaning) = act (root word)

In [30]:
port_stem = PorterStemmer()

In [31]:
def stemming(content): # here 'content' is the 'text' col from the dataset.

  stemmed_content = re.sub('[^a-zA-Z]', ' ', content) # here we are removing all the characters which are not (a-z) & (A-Z)
  # It is used to remove all the  punctuations and other symbols from the given text, which are not alphabets

  stemmed_content = stemmed_content.lower() # converting to lowercase

  stemmed_content = stemmed_content.split() # splitting the words from the processed text to form a list

  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  # here we are using the 'stopwords', means we are checking whether the 'stemmed_content' contains the stopwords or not
  # If the 'stemmed_content' doesn't contains any stopwords,
  # then the stemming function is applied over it to reduce the word to its root word

  stemmed_content = ' '.join(stemmed_content)
  # after the processing, we are again joining those words together

  return stemmed_content

In [32]:
twitter_data['stemmed_content'] = twitter_data['text'].apply(stemming)

In [33]:
temp_df = twitter_data['stemmed_content']

In [34]:
print(twitter_data['stemmed_content'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_content, Length: 1600000, dtype: object


In [35]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [61]:
# Separating the data and label
X = twitter_data['stemmed_content'].values
Y = twitter_data['target'].values

In [37]:
print(X)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [38]:
print(Y)

[0 0 0 ... 1 1 1]


# Splitting the data to training data & test data

In [62]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y, random_state = 2)

## **test_size = 0.2**

'test_size = 0.2' means 20% percent of data will go for test data and remaining 80% of data will go to training data

## **stratify = Y**

we are performing stratify on the data 'Y' because we need equal proportion of 2 classes (0 & 1) to go to both training & test data.

we don't want it to be like all positive (1) go to training and all the negatives (0) to go to test data

## **random_state = 2**

It is used to separate the data in a random manner

In [40]:
print(X.shape, X_train.shape, X_test.shape)

(1600000,) (1280000,) (320000,)


In [41]:
print(Y.shape, Y_train.shape, Y_test.shape)

(1600000,) (1280000,) (320000,)


# Converting the textual data to numerical data

converting the text to Numerical Values for ML model to understand ,
here we are going to use a process called "Feature Extraction"

## using the (vectorizer.fit_transform()) process each word in the training and test data will be given a weight/importance based on how many times it has been repeated, then the model will understand the sentiment of the word using this method.
Example : if a word 'happy' is present 5 times and word 'fine' is present 2 times, then (vectorizer.fit_transform()) method will assign it 5 and 2 respectively. while training the model will use these numerical values (5,2) to understand that the word happy means more positive and fine means less positive

In [64]:
vectorizer = TfidfVectorizer()

# Use the properly split training and test data for vectorization
X_train_vectorized = vectorizer.fit_transform(X_train)  # Fit and transform training data
X_test_vectorized = vectorizer.transform(X_test)        # Only transform test data (don't fit)

# Save the vectorizer
import pickle
with open("vectorizer.sav", "wb") as file:
    pickle.dump(vectorizer, file)

In [63]:
print(X_train)

['watch saw iv drink lil wine' 'hatermagazin'
 'even though favourit drink think vodka coke wipe mind time think im gonna find new drink'
 ... 'eager monday afternoon'
 'hope everyon mother great day wait hear guy store tomorrow'
 'love wake folger bad voic deeper']


In [49]:
print(X_test)

['mmangen fine much time chat twitter hubbi back summer amp tend domin free time'
 'ah may show w ruth kim amp geoffrey sanhueza'
 'ishatara mayb bay area thang dammit' ...
 'destini nevertheless hooray member wonder safe trip' 'feel well'
 'supersandro thank']


# **Training the Machine Learning Model**

**Logistic Regression**

In [50]:
model = LogisticRegression(max_iter = 1000)

In [65]:
model.fit(X_train_vectorized, Y_train)
# X_train_vectorized is the vectorized training data tweets
# Y_train is the target for the training data tweets

LogisticRegression(max_iter=1000)

# **Model Evaluation**

AccuracyScore : we are going to use AccuracyScore for Evaluating the accuracy of the model

In [66]:
X_train_prediction = model.predict(X_train_vectorized) # here the trained model predicts the output for X_train_vectorized data
training_data_accuracy = accuracy_score(Y_train, X_train_prediction) # we are now comparing the prediction with the true labels

In [67]:
print("Accuracy score on the training data :", training_data_accuracy)

Accuracy score on the training data : 0.79871953125


# Testing the accuracy on the test data

In [68]:
X_test_prediction = model.predict(X_test_vectorized)
test_data_accuracy = accuracy_score(Y_test, X_test_prediction)

In [69]:
print("Accuracy score on the tes data :", test_data_accuracy)
# here our model has performed well as the accuracy of training data and of the test data is very close to each other

Accuracy score on the tes data : 0.77668125


## Overfitting occurs when the accuracy of the training data is very high while accuracy of test data is very low

Model Accuracy = 77.6 %

# Exporting the Model

In [70]:
import pickle

In [71]:
filename = 'trained_model.sav'
pickle.dump(model, open(filename, 'wb'))

# Using the Saved Model for future Predictions

In [73]:
# loading the saved model
loaded_model = pickle.load(open('trained_model.sav', 'rb'))

In [74]:
X_new = X_test[200]
print(Y_test[200])

1


In [76]:
X_new_vectorized = vectorizer.transform([X_new])
prediction = loaded_model.predict(X_new_vectorized)
print(prediction)

if prediction[0] == 0:
    print("Negative Tweet")
else:
    print("Positive Tweet")

[1]
Positive Tweet
